## Score Dataset 3 with all SFs

In [19]:
import pandas as pd
import HLA_Arena as arena
import os
import pyrosetta
import pickle as pk
import numpy as np

## Load the dataset

In [14]:
folder_src = "/data/COMP590P/energy_analysis/Fixed_PDBs/"

dataset = pd.read_csv("../Datasets/dataset3.csv")
dataset = dataset[["peptide", "pdb_code", "BA", "fname", "complex_number", "alt_conf", "resolution"]]
dataset

,peptide,pdb_code,BA,fname,complex_number,alt_conf,resolution
0,AAGIGILTV,3QFD,395,3QFD-2b_complex.pdb,2,b,1.68
1,LLFGYPVYV,4E5X,3.8,4E5X-1_complex.pdb,1,a,1.95
2,ILKEPVHGV,2X4U,192.3,2X4U-2_complex.pdb,2,a,2.1
3,LLFGYPVYV,2AV1,3.8,2AV1-2_complex.pdb,2,a,1.95
4,LLFGYPVYV,5IRO,3.8,5IRO-2O_complex.pdb,2,a,2.64
...,...,...,...,...,...,...,...
77,LLFGYPVYV,5IRO,3.8,5IRO-6O_complex.pdb,6,a,2.64
78,AAGIGILTV,2GUO,395,2GUO-1_complex.pdb,1,a,1.9
79,LLFGYPVYV,5IRO,3.8,5IRO-1O_complex.pdb,1,a,2.64
80,SLLMWITQC,2P5W,21070,2P5W-1_complex.pdb,1,a,2.2


## Score with 3pHLA

In [37]:
# util functions for trating the data
def init_rosetta():
    pyrosetta.init()
    
def get_energies(X):
    ene = np.roll(X, 4, axis = 0)[:9,:19]
    ene = np.roll(ene, -4, axis = 0)
    ene = ene.reshape(9*19)
    return ene


def extract_ppp_energies(file_name, pep_len):
    print(file_name)
    print(pep_len)
    scorefxn=pyrosetta.get_fa_scorefxn()
    #load rosetta
    pose = pyrosetta.pose_from_pdb(file_name)
    scorefxn(pose)
    #get energies
    res_ene = pose.energies().residue_total_energies_array()
    peptide_ene = res_ene[-pep_len:]
    pep_ene = [np.array(list(x)) for x in peptide_ene]
    pep_ene = np.array(pep_ene)
    return get_energies(pep_ene)


def score_ppp_hla(file_name, pep_len, rfModel):
    energies = extract_ppp_energies(file_name, pep_len)
    print(len(energies))
    return rfModel.predict([energies])

def convert_nM(pred_val):
    return math.exp((1-pred_val)*math.log(50000))


In [38]:
init_rosetta()
allele="A0201"
root_dir = "../Experiment1 - train ref2015-score, standard-pHLA-score and 3pHLA-score"
model_root = root_dir+"/final_REGRmodels/"
model_9 = pk.load(open(model_root+"A0201ppp.pkl", 'rb'))

dataset["3pHLA-score"] = dataset.apply(lambda row: score_ppp_hla(folder_src+row["fname"], len(row["peptide"]), model_9), axis=1)

PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python36.linux 2021.07+release.c48be2695c4ba637c6fa19ee5d289fd9a8aa99ef 2021-02-21T11:50:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
/data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-2O_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/NA6P_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/2GT9-2_complex.pdb
10
171
/data/COMP590P/energy_analysis/Fixed_PDBs/4FTV-1_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-1_complex.pdb
9
171
/data/COMP590P/energy_analysis/Fixed_PDBs/4E5X

## Score with Vina, Vinardo, AutoDock

In [39]:
import os.path

funcs = ["vina", "vinardo", "ad4_scoring"]
results = {"filename":[], "peptide":[], "pdb_id":[], "energy":[], "func": []}

def score_HLAArena(folder_src, func, row):
    energy = arena.rescore_complex_simple_smina(folder_src+row["fname"], func)
    print("File "+folder_src+row["fname"]+ " scored by "+func)
    print("Score =  "+str(energy))
    return energy

for func in funcs:
    dataset[func+"-score"] = dataset.apply(lambda row: score_HLAArena(folder_src, func, row), axis=1)


File /data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb scored by vina
Score =  -7.66833
File /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb scored by vina
Score =  -8.86559
File /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb scored by vina
Score =  -7.59544
File /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb scored by vina
Score =  -9.12007
File /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-2O_complex.pdb scored by vina
Score =  -5.1094
File /data/COMP590P/energy_analysis/Fixed_PDBs/NA6P_complex.pdb scored by vina
Score =  -7.00214
File /data/COMP590P/energy_analysis/Fixed_PDBs/2GT9-2_complex.pdb scored by vina
Score =  -8.30754
File /data/COMP590P/energy_analysis/Fixed_PDBs/4FTV-1_complex.pdb scored by vina
Score =  -9.93841
File /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-1_complex.pdb scored by vina
Score =  -9.18351
File /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-2_complex.pdb scored by vina
Score =  -9.89666
File /data/

File /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb scored by vinardo
Score =  -12.61647
File /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb scored by vinardo
Score =  -9.72807
File /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb scored by vinardo
Score =  -13.3307
File /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-2O_complex.pdb scored by vinardo
Score =  -6.6063
File /data/COMP590P/energy_analysis/Fixed_PDBs/NA6P_complex.pdb scored by vinardo
Score =  -10.10812
File /data/COMP590P/energy_analysis/Fixed_PDBs/2GT9-2_complex.pdb scored by vinardo
Score =  -11.40464
File /data/COMP590P/energy_analysis/Fixed_PDBs/4FTV-1_complex.pdb scored by vinardo
Score =  -14.31767
File /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-1_complex.pdb scored by vinardo
Score =  -13.08367
File /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-2_complex.pdb scored by vinardo
Score =  -14.32921
File /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-1a_complex.pdb scored by v

File /data/COMP590P/energy_analysis/Fixed_PDBs/2F54-1_complex.pdb scored by vinardo
Score =  -9.64558
File /data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb scored by ad4_scoring
Score =  -50.91808
File /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb scored by ad4_scoring
Score =  -52.03132
File /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb scored by ad4_scoring
Score =  -59.62396
File /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb scored by ad4_scoring
Score =  -60.26314
File /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-2O_complex.pdb scored by ad4_scoring
Score =  -23.62013
File /data/COMP590P/energy_analysis/Fixed_PDBs/NA6P_complex.pdb scored by ad4_scoring
Score =  -54.72781
File /data/COMP590P/energy_analysis/Fixed_PDBs/2GT9-2_complex.pdb scored by ad4_scoring
Score =  -73.39983
File /data/COMP590P/energy_analysis/Fixed_PDBs/4FTV-1_complex.pdb scored by ad4_scoring
Score =  -61.3685
File /data/COMP590P/energy_analysis/Fixed_P

File /data/COMP590P/energy_analysis/Fixed_PDBs/2F54-2_complex.pdb scored by ad4_scoring
Score =  -49.01867
File /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-6O_complex.pdb scored by ad4_scoring
Score =  -43.20399
File /data/COMP590P/energy_analysis/Fixed_PDBs/2GUO-1_complex.pdb scored by ad4_scoring
Score =  -36.68002
File /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-1O_complex.pdb scored by ad4_scoring
Score =  -16.61786
File /data/COMP590P/energy_analysis/Fixed_PDBs/2P5W-1_complex.pdb scored by ad4_scoring
Score =  -56.03613
File /data/COMP590P/energy_analysis/Fixed_PDBs/2F54-1_complex.pdb scored by ad4_scoring
Score =  -52.95511


## Score with DOPE

In [42]:
from modeller import *
from modeller.automodel import *
from modeller.scripts import complete_pdb

In [43]:
def score_dope(folder_src, row):
    #score    
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    
    mdl = complete_pdb(env, folder_src+row["fname"])
        
    atmsel_lig = selection(mdl.chains[2])
    atmsel_rec = selection(mdl.chains[0], mdl.chains[1])
    atmsel_full = selection(mdl.chains[0], mdl.chains[1], mdl.chains[2])
    
    score_lig = atmsel_lig.assess_dope()
    score_rec = atmsel_rec.assess_dope()
    score_full = atmsel_full.assess_dope()
    
    score_diff = score_full - (score_rec + score_lig) 
    
    return (score_full, score_diff)

In [44]:
dataset["dope_score"] = dataset.apply(lambda row: score_dope(folder_src, row), axis=1)


                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux b233047cffab 4.15.0-142-generic x86_64
Date and time of compi



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      384
Number of all, selected real atoms                :     3161    3084
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   630366
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMI

read_to_681_> topology.submodel read from topology file:        3
>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  GLU:OXT  GLU
iatmcls_286W> MODEL atom not classified:  MET:OXT  MET
iatmcls_286W> MODEL atom not classified:  VAL:OXT  VAL
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      384
Number of all, selected real atoms                :     3161      77
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):    32849
Dynamic pairs routine                             : 5, N

>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  GLU:OXT  GLU
iatmcls_286W> MODEL atom not classified:  MET:OXT  MET
iatmcls_286W> MODEL atom not classified:  VAL:OXT  VAL
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      385
Number of all, selected real atoms                :     3150      66
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):    30439
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDAT



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      384
Number of all, selected real atoms                :     3161    3161
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   621925
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMIC_PA

...



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      385
Number of all, selected real atoms                :     3168    3085
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   636200
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMI

>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  TRP:OXT  TRP
iatmcls_286W> MODEL atom not classified:  MET:OXT  MET
iatmcls_286W> MODEL atom not classified:  CYS:OXT  CYS
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      383
Number of all, selected real atoms                :     3141      75
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):    32128
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDAT

>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  GLU:OXT  GLU
iatmcls_286W> MODEL atom not classified:  MET:OXT  MET
iatmcls_286W> MODEL atom not classified:  VAL:OXT  VAL
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      384
Number of all, selected real atoms                :     3141      57
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):    26167
Dynamic pairs routine                             : 5, NATM_SEL x NATM double loop
Atomic shift for contacts update (UPDAT

read_to_681_> topology.submodel read from topology file:        3
>> Model assessment by DOPE potential
iatmcls_286W> MODEL atom not classified:  PRO:OXT  PRO
iatmcls_286W> MODEL atom not classified:  MET:OXT  MET
iatmcls_286W> MODEL atom not classified:  CYS:OXT  CYS
preppdf_453W> No fixed restraints selected; there may be some dynamic ones.
preppdf_454W> Restraints file was probably not read; use restraints.append().


>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      385
Number of all, selected real atoms                :     3163      75
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):    31680
Dynamic pairs routine                             : 5, N



>> ENERGY; Differences between the model's features and restraints:
Number of all residues in MODEL                   :      383
Number of all, selected real atoms                :     3141    3141
Number of all, selected pseudo atoms              :        0       0
Number of all static, selected restraints         :        0       0
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   620003
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMIC_PA

In [47]:
pd.read_csv("dataset3_tmpres.csv")

,Unnamed: 0,peptide,pdb_code,BA,fname,complex_number,alt_conf,resolution,3pHLA-score,vina-score,vinardo-score,ad4_scoring-score,dope_score
0,0,AAGIGILTV,3QFD,395,3QFD-2b_complex.pdb,2,b,1.68,[0.33159973],-7.66833,-9.81796,-50.91808,"(-45332.65234375, 1362.5146484375)"
1,1,LLFGYPVYV,4E5X,3.8,4E5X-1_complex.pdb,1,a,1.95,[0.56449166],-8.86559,-12.61647,-52.03132,"(-45189.19140625, 1972.84912109375)"
2,2,ILKEPVHGV,2X4U,192.3,2X4U-2_complex.pdb,2,a,2.1,[0.56597111],-7.59544,-9.72807,-59.62396,"(-44696.671875, 1371.7520751953125)"
3,3,LLFGYPVYV,2AV1,3.8,2AV1-2_complex.pdb,2,a,1.95,[0.54672185],-9.12007,-13.33070,-60.26314,"(-45773.51953125, 2029.514404296875)"
4,4,LLFGYPVYV,5IRO,3.8,5IRO-2O_complex.pdb,2,a,2.64,[0.56909853],-5.10940,-6.60630,-23.62013,"(-37164.16796875, 1827.47119140625)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,LLFGYPVYV,5IRO,3.8,5IRO-6O_complex.pdb,6,a,2.64,[0.56388838],-5.66112,-6.38925,-43.20399,"(-35238.41796875, 1854.300537109375)"
78,78,AAGIGILTV,2GUO,395,2GUO-1_complex.pdb,1,a,1.9,[0.34409796],-7.40784,-9.89774,-36.68002,"(-45112.5703125, 1405.6510009765625)"
79,79,LLFGYPVYV,5IRO,3.8,5IRO-1O_complex.pdb,1,a,2.64,[0.5619652],-5.29907,-6.72671,-16.61786,"(-36893.703125, 1883.082275390625)"
80,80,SLLMWITQC,2P5W,21070,2P5W-1_complex.pdb,1,a,2.2,[0.54803375],-6.76755,-9.16736,-56.03613,"(-44858.078125, 1608.296630859375)"


## Score with FoldX

In [50]:
%cd ..

/data/3pHLA-training


In [51]:
from modeller import *
from modeller.automodel import *
from modeller.scripts import complete_pdb

In [59]:
import subprocess
import subprocess

def find_pHLA(file_path):
    if not "rdf_mount" in file_path:
        return file_path[file_path.rfind("/")+1:file_path.find(".pdb")]
    else:
        prefix = "../../rdf_mount/decoymulticonf/confs/"
        suffix = "/full_system_confs"
        return file_path[file_path.find(prefix)+len(prefix):file_path.find(suffix)]
    

def score_FX(row):
    print("processing "+row["path"])
    #score    
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    
    
    mdl = complete_pdb(env, row["path"])
    chain_rec = mdl.chains[0].name
    chain_pep = mdl.chains[2].name
    
    cname = find_pHLA(row["path"])
    command = "./script_rescore_FoldX.sh "+chain_pep+" "+chain_rec+" "+row["path"]+" "+cname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    ret = process.returncode
    print(ret)
    if not ret == 0: print("Error with execution") 
    print("done with "+row["path"])
    

In [58]:
process = subprocess.Popen("touch FoldX_results.csv", shell=True, stdout=subprocess.PIPE)
process = subprocess.Popen("echo \"name, ene\" > FoldX_results.csv", shell=True, stdout=subprocess.PIPE)
process.wait()
process.returncode

0

In [65]:
dataset["path"] = dataset["fname"].apply(lambda x: "/data/COMP590P/energy_analysis/Fixed_PDBs/"+x)
dataset.apply(lambda row: score_FX(row), axis=1)

processing /data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/3QFD-2b_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/4E5X-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2X4U-2_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2AV1-2_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-2O_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /da

0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/5IVX-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2P5E-1a_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2P5E-1a_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/1DDH-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/1DDH-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/3OXS-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/3OXS-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/1P7Q-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/1P7Q-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2AV7-2_complex.pdb
read_to_6

0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2GUO-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-1O_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/5IRO-1O_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2P5W-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2P5W-1_complex.pdb
processing /data/COMP590P/energy_analysis/Fixed_PDBs/2F54-1_complex.pdb
read_to_681_> topology.submodel read from topology file:        3
0
done with /data/COMP590P/energy_analysis/Fixed_PDBs/2F54-1_complex.pdb


0     None
1     None
2     None
3     None
4     None
      ... 
77    None
78    None
79    None
80    None
81    None
Length: 82, dtype: object

In [66]:
FX_res = pd.read_csv("FoldX_results.csv")
FX_res["fname"] = FX_res["name"].apply(lambda x: x+".pdb")  

def extract_FX_ene(row, FX_res):
    matchl = FX_res[FX_res["fname"]==row["fname"]]
    if len(matchl) == 0 : return (None, None)
    return float(list(matchl[" ene"])[0])

dataset["foldx_score"] = dataset.apply(lambda row: extract_FX_ene(row, FX_res), axis=1)

In [69]:
%cd ./Experiment3 - crystal structures etc

/data/3pHLA-training/Experiment3 - crystal structures etc


In [71]:
dataset.to_csv("dataset3_tmpres.csv")

## Score with GradDock

In [ ]:
from modeller import *
from modeller.automodel import *
from modeller.scripts import complete_pdb

In [ ]:
import subprocess
## Access ligand/receptor files

def extract_receptor_ligand(filename):
    env = environ()
    env.libs.topology.read(file='$(LIB)/top_heav.lib')
    env.libs.parameters.read(file='$(LIB)/par.lib')
    mdl = complete_pdb(env, filename)
    
    atmsel_lig = selection(mdl.chains[2])
    atmsel_rec = selection(mdl.chains[0], mdl.chains[1])
    
    lig_name = filename[:filename.find(".pdb")]+"_ligand.pdb"
    rec_name = filename[:filename.find(".pdb")]+"_receptor.pdb"
    atmsel_lig.write(lig_name)
    atmsel_rec.write(rec_name)
    
    return (lig_name, rec_name)
    
def get_rec_lig_name(path):
    
    if "singleconf" in path:
        lname = lfname[lfname.rfind("/")+1:]
        rname = rfname[rfname.rfind("/")+1:]
        return ()
    
def score_GD(row):
    print("processing "+row["path"])
    (lfname, rfname) = extract_receptor_ligand(row["path"])
    folder_src = row["path"][:row["path"].rfind("/")]
    lname = lfname[lfname.rfind("/")+1:]
    rname = rfname[rfname.rfind("/")+1:]
    command = "./GD_run.sh "+lname+" "+rname+" "+folder_src+" > GD_progress.txt"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    command = "rm "+lfname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    command = "rm "+rfname
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    process.wait()
    ret = process.returncode
    print(ret)
    if not ret == 0: print("Error with execution") 
    print("done with "+row["path"])
    

In [ ]:
%cd /graddock/GD/evaluate
%pwd

In [ ]:
process = subprocess.Popen("touch GD_results.csv", shell=True, stdout=subprocess.PIPE)
process = subprocess.Popen("echo \"name, complex, diff\" GD_results.csv", shell=True, stdout=subprocess.PIPE)
process.wait()
process.returncode

In [ ]:
dataset.apply(lambda row: score_GD(row), axis=1)

In [ ]:
GD_res = pd.read_csv("GD_results.csv")
GD_res["fname"] = GD_res["name"].apply(lambda x: x[:-9]+"_complex.pdb")  

def extract_GD_ene(row, GD_res):
    matchl = GD_res[GD_res["fname"]==row["fname"]]
    if len(matchl) == 0: return (None, None)
    return (float(list(matchl[" complex"])[0]),float(list(matchl[" binding"])[0]))

dataset["GD_score"] = dataset.apply(lambda row: extract_GD_ene(row, GD_res), axis=1)

In [81]:
dataset.to_csv("dataset3_results.csv")